In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
sub = 'pilot-52'
ses = 'ses-01'
battery = 'camcan'
task_source = 'gng'
task = 'stop-nogo'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery)
csv_file = task_source + '_sub-52_' + ses + '.txt'
tsv_file = task + '_sub-52_' + ses + '.tsv'

In [8]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_file), sep='\t', skipinitialspace=True)

In [58]:
file_df

,tr#,type,arrLR,Stype,Sdel,butt,resp,resp2,RT,RT2,RTsum,eval,relStart
0,1,0,2,0,0,2,2,0,395,0,0,1,0
1,2,1,2,2,250,2,0,2,0,73,323,2,2522
2,3,0,2,0,0,2,2,0,355,0,0,1,5068
3,4,0,1,0,0,1,1,0,256,0,0,1,7591
4,5,0,1,0,0,1,1,0,284,0,0,1,10113
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,236,0,1,0,0,1,1,0,275,0,0,1,594226
236,237,0,2,0,0,2,2,0,353,0,0,1,596748
237,238,0,2,0,0,2,2,0,406,0,0,1,599271
238,239,0,1,0,0,1,1,0,286,0,0,1,601794


In [59]:
file_df.keys()

Index(['tr#', 'type', 'arrLR', 'Stype', 'Sdel', 'butt', 'resp', 'resp2', 'RT',
       'RT2', 'RTsum', 'eval', 'relStart'],
      dtype='object')

In [9]:
# Extract the onsets of stimulus, probe and response
trial_onsets = (file_df['relStart'] + 500)

# Find the 4 trials types:
# 1. Go trials ('type' = 0)
# 2. No-go trials ('type' = 1 and 'Stype' = 1)
# 3. Successful stop trials ('type' = 1 and 'Stype' = 2 or 3 and RT1 = 0 and RT2 = 0)
# 3. Unsuccessful stop trials ('type' = 1 and 'Stype' = 2 or 3 and (RT1 != 0 or RT2 != 0))
go_trials = file_df['type'] == 0
unsucgo_trials = file_df.loc[go_trials][file_df['RT'] == 0].index
nogo_trials = (file_df['type'] == 1) & (file_df['Stype'] == 1)
unsucnogo_trials = file_df.loc[nogo_trials][file_df['RT'] != 0].index
sucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] == 0)
unsucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] != 0)

# Calculate trial durations
durations = np.array(np.ones(len(trial_onsets)), dtype=np.int_)
durations[go_trials] = file_df.loc[go_trials]['RT']
durations[unsucgo_trials] = 2500
durations[nogo_trials] = 2500
durations[unsucnogo_trials] = file_df.loc[unsucnogo_trials]['RT']
durations[sucstop_trials] = 2500
durations[unsucstop_trials] = file_df.loc[unsucstop_trials]['RTsum']

# Assign trial types
trial_types = np.array(np.ones(len(trial_onsets)), dtype=np.object_)
trial_types[go_trials] = 'go'
trial_types[nogo_trials] = 'nogo'
trial_types[sucstop_trials] = 'successful_stop'
trial_types[unsucstop_trials] = 'unsuccessful_stop'

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [61]:
durations

array([ 395,  323,  355,  256,  284,  320,  306,  374,  394,  281,  348,
        293,  436,  231,  265,  355,  383,  339,  336,  356, 2500,  344,
        363,  287,  299,  335, 2500,  427,  409,  365,  382,  358,  394,
        348,  398,  316,  330,  364,  310,  311,  323,  317,  293,  356,
       2500,  300,  516,  250,  355,  286,  583,  384, 2500, 2500, 2500,
        403,  429,  466,  619, 2500,  404, 2500, 2500, 2500,  303, 2500,
        340,  326, 2500, 2500,  382,  434,  274,  369,  354,  413,  393,
        359,  387,  327,  268,  281,  284,  354,  308,  349, 2500,  307,
        242,  397,  382,  389,  588,  307,  558,  239,  401,  401,  378,
       2500,  470,  518,  481,  283,  383,  282,  353,  282,  344,  365,
        217,  280,  303, 2500,  295, 2500,  277,  339,  282,  316,  337,
        256,  376,  276, 2500,  337,  330,  502,  370,  367,  364,  323,
        352,  383,  353, 2500,  416,  514,  368, 2500, 2500,  416, 2500,
        400,  363,  557,  364,  395,  313,  256,  2

In [10]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':trial_onsets/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)